# **PART 2. DATA PREPROCESSING**

## **1. Overview of preprocessing and data exploration**
- Check the initial data.
- Clean the initial data.
- Merge the files together.
- Process the data after merging.
- Print out the file movie.csv


## **2. Read the original obtained data file**


### Import libraries

In [1]:
#import library
import requests
import numpy as np
import pandas as pd

In [2]:
#Read data
IMDB = pd.read_csv('imdb.csv')
TMDB = pd.read_csv('TMDB.csv')
ROTTEN = pd.read_csv('rotten.csv')

In [3]:
print(IMDB.head())

                      Title  imdbRating
0  The Shawshank Redemption         9.3
1             The Godfather         9.2
2           The Dark Knight         9.0
3     The Godfather Part II         9.0
4              12 Angry Men         9.0


In [4]:
print (TMDB.head())

                                         Poster_Link  \
0  https://image.tmdb.org/t/p/w500/9cqNxx0GxF0bfl...   
1  https://image.tmdb.org/t/p/w500/3bhkrj58Vtu7en...   
2  https://image.tmdb.org/t/p/w500/hek3koDUyRQk7F...   
3  https://image.tmdb.org/t/p/w500/sF1U4EUQS8YHUY...   
4  https://image.tmdb.org/t/p/w500/ow3wq89wM8qd5X...   

                      Title  \
0  The Shawshank Redemption   
1             The Godfather   
2     The Godfather Part II   
3          Schindler's List   
4              12 Angry Men   

                                            Overview Certificate  \
0  Imprisoned in the 1940s for the double murder ...           R   
1  Spanning the years 1945 to 1955, a chronicle o...           R   
2  In the continuing saga of the Corleone crime f...           R   
3  The true story of how businessman Oskar Schind...           R   
4  The defense and the prosecution have rested an...          NR   

   Runtime (min)                Genre  \
0            142         D

In [5]:
print (ROTTEN.head())

               Title  rottenRating
0      The Godfather            97
1         Casablanca            99
2  L.A. Confidential            99
3      Seven Samurai           100
4           Parasite            99


In [6]:
# # Loại bỏ các dấu ngoặc trong cột imdbRating của của DataFrame IMDB
# IMDB['imdbRating'] = IMDB['imdbRating'].str.replace('[(),]', '', regex=True).astype(float)
# # Chuyển đổi kiểu dữ liệu của cột imdbRating thành float    
# IMDB['imdbRating'] = IMDB['imdbRating'].astype(float)
# # Lưu lại vào file CSV 
# IMDB.to_csv('imdb.csv', index=False)

In [7]:
print (IMDB.head())

                      Title  imdbRating
0  The Shawshank Redemption         9.3
1             The Godfather         9.2
2           The Dark Knight         9.0
3     The Godfather Part II         9.0
4              12 Angry Men         9.0


## **3.Merge the initial files together.**

In [8]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Đọc dữ liệu
df_a = pd.read_csv('TMDB.csv')  # A: Title + 8 cột
df_b = pd.read_csv('imdb.csv')  # B: Title + 1 cột dữ liệu
df_c = pd.read_csv('rotten.csv')  # C: Title + 1 cột dữ liệu

# Lấy danh sách tiêu đề từ B và C
titles_b = df_b['Title'].tolist()
titles_c = df_c['Title'].tolist()

# Hàm fuzzy match
def match_title(title, choices, threshold=90):
    match, score = process.extractOne(title, choices, scorer=fuzz.token_set_ratio)
    return match if score >= threshold else None

# Ghép tiêu đề tương ứng từ B và C (fuzzy matched)
df_a['Matched_B'] = df_a['Title'].apply(lambda t: match_title(t, titles_b))
df_a['Matched_C'] = df_a['Title'].apply(lambda t: match_title(t, titles_c))

# Lấy tên cột dữ liệu duy nhất ngoài Title trong B và C
col_b = [col for col in df_b.columns if col != 'Title'][0]
col_c = [col for col in df_c.columns if col != 'Title'][0]

# Ghép cột từ B
merged = pd.merge(
    df_a, df_b[['Title', col_b]],
    left_on='Matched_B', right_on='Title',
    how='left'
)



# Ghép cột từ C
merged = pd.merge(
    merged, df_c[['Title', col_c]],
    left_on='Matched_C', right_on='Title',
    how='left'
)


# Xóa các cột phụ không cần giữ
merged.drop(columns=['Matched_B', 'Matched_C', 'Title_y', 'Title'], inplace=True)

# Đặt lại tên cột Title gốc (Title_x)
merged.rename(columns={'Title_x': 'Title'}, inplace=True)

# Xuất kết quả
merged.to_csv('merged_result.csv', index=False)
print(merged.head())

data = merged

                                         Poster_Link  \
0  https://image.tmdb.org/t/p/w500/9cqNxx0GxF0bfl...   
1  https://image.tmdb.org/t/p/w500/3bhkrj58Vtu7en...   
2  https://image.tmdb.org/t/p/w500/hek3koDUyRQk7F...   
3  https://image.tmdb.org/t/p/w500/sF1U4EUQS8YHUY...   
4  https://image.tmdb.org/t/p/w500/ow3wq89wM8qd5X...   

                      Title  \
0  The Shawshank Redemption   
1             The Godfather   
2     The Godfather Part II   
3          Schindler's List   
4              12 Angry Men   

                                            Overview Certificate  \
0  Imprisoned in the 1940s for the double murder ...           R   
1  Spanning the years 1945 to 1955, a chronicle o...           R   
2  In the continuing saga of the Corleone crime f...           R   
3  The true story of how businessman Oskar Schind...           R   
4  The defense and the prosecution have rested an...          NR   

   Runtime (min)                Genre  \
0            142         D

## **4.Process the data after merging.**

## Does the raw data have duplicate rows?

In [9]:
# Check if data have duplicate rows
num_duplicated_rows = data.duplicated().sum()
if num_duplicated_rows > 0:
    print(f"Found {num_duplicated_rows} duplicated rows. Removing duplicates...")
    data = data.drop_duplicates()
    print("✅ Duplicates removed.")
else:
    print("✅ Your data has no duplicated rows.")

✅ Your data has no duplicated rows.


## What data type does each column currently have? Are there any columns whose data types are not suitable for further processing?

In [10]:
#Type of each column
dtypes = data.dtypes
print("Data types of each column:")
print(dtypes)

Data types of each column:
Poster_Link       object
Title             object
Overview          object
Certificate       object
Runtime (min)      int64
Genre             object
Actors            object
Director          object
Year               int64
tmdbRating       float64
imdbRating       float64
rottenRating     float64
dtype: object


## Check the percentage of missing data in the columns.

In [11]:
#Percentage of missing data
missing_percentage = data.isnull().mean() * 100
print("Missing ratio")
print(missing_percentage)

Missing ratio
Poster_Link       0.0
Title             0.0
Overview          0.0
Certificate      17.8
Runtime (min)     0.0
Genre             0.0
Actors            0.8
Director          0.0
Year              0.0
tmdbRating        0.0
imdbRating       57.4
rottenRating     69.0
dtype: float64


- After identifying the basic statistical numbers that describe data, we further need to determine the features that have a large number of missing values. Such features are not useful for the analysis stage and must be removed from the dataset.

- Depending on goals, the threshold for "large" can be defined. Usually, if the percentage of missing values is greater than 75%, the column is dropped from the dataframe and an updated dataframe is returned.

In [12]:
def drop_missing_features(df: pd.DataFrame, missing_percentage: pd.Series, threshold: float = 75.0) -> pd.DataFrame:
    # Find columns with missing data percentage greater than the threshold
    cols_to_drop = missing_percentage[missing_percentage > threshold].index
    # Drop those columns from the DataFrame
    return df.drop(columns=cols_to_drop)

# Apply the function to the dataframe `data` using the `missing_percentage` series
data = drop_missing_features(data, missing_percentage)

# Display the first few rows of the resulting dataframe
data.head()


,Poster_Link,Title,Overview,Certificate,Runtime (min),Genre,Actors,Director,Year,tmdbRating,imdbRating,rottenRating
0,https://image.tmdb.org/t/p/w500/9cqNxx0GxF0bfl...,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,R,142,"Drama, Crime","Morgan Freeman, Tim Robbins, Bob Gunton, Willi...",Frank Darabont,1994,8.700,9.3,NaN
1,https://image.tmdb.org/t/p/w500/3bhkrj58Vtu7en...,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",R,175,"Drama, Crime","Marlon Brando, Al Pacino, James Caan, Robert D...",Francis Ford Coppola,1972,8.686,9.2,97.0
2,https://image.tmdb.org/t/p/w500/hek3koDUyRQk7F...,The Godfather Part II,In the continuing saga of the Corleone crime f...,R,202,"Drama, Crime","Al Pacino, Robert Duvall, Diane Keaton, Robert...",Francis Ford Coppola,1974,8.570,9.2,97.0
3,https://image.tmdb.org/t/p/w500/sF1U4EUQS8YHUY...,Schindler's List,The true story of how businessman Oskar Schind...,R,195,"Drama, History, War","Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",Steven Spielberg,1993,8.564,9.0,98.0
4,https://image.tmdb.org/t/p/w500/ow3wq89wM8qd5X...,12 Angry Men,The defense and the prosecution have rested an...,NR,97,Drama,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",Sidney Lumet,1957,8.548,9.0,100.0


##### After determining the percentage of missing data in the columns, we will now **divide them into two categories: numeric data type and non-numeric data type** for processing.

## For each column with numeric data type, how are the values distributed

For columns with numeric data types, we will calculate:
- Percentage (from 0 to 100) of missing values
- The min
- The lower quartile
- The median
- The upper quartile
- The max



In [13]:
def missing_ratio(column):
    return column.isna().mean() * 100

def lower_quartile(column):
    return column.quantile(0.25)

def median(column):
    return column.median()

def upper_quartile(column):
    return column.quantile(0.75)

# Select numerical columns (float64, int64) from the DataFrame
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns

# Dictionary to store the statistics
statistics = {
    "missing_ratio": [],
    "min": [],
    "lower_quartile": [],
    "median": [],
    "upper_quartile": [],
    "max": []
}

# Calculate statistics for each numerical column
for col in numeric_cols:
    missing_ratio_val = missing_ratio(data[col])
    min_val = data[col].min()
    lower_quartile_val = lower_quartile(data[col])
    median_val = median(data[col])
    upper_quartile_val = upper_quartile(data[col])
    max_val = data[col].max()
    
    statistics["missing_ratio"].append(missing_ratio_val)
    statistics["min"].append(min_val)
    statistics["lower_quartile"].append(lower_quartile_val)
    statistics["median"].append(median_val)
    statistics["upper_quartile"].append(upper_quartile_val)
    statistics["max"].append(max_val)

# Create a DataFrame from the statistics dictionary
num_col_info_df = pd.DataFrame(statistics, index=numeric_cols).T.round(1)
print(num_col_info_df)


                Runtime (min)    Year  tmdbRating  imdbRating  rottenRating
missing_ratio             0.0     0.0         0.0        57.4          69.0
min                       4.0  1902.0         7.9         8.1          89.0
lower_quartile          100.0  1975.8         7.9         8.2          94.0
median                  117.0  2004.0         8.0         8.3          96.0
upper_quartile          137.0  2018.0         8.2         8.5          98.0
max                     367.0  2025.0         8.7         9.3         100.0


## For each column with a non-numeric data type, how are the values distributed?

In [14]:
non_numeric_cols = data.select_dtypes(exclude=['float64', 'int64']).columns
cat_statistics = {
    "missing_ratio": [],
    "num_values": [],  #Numbers of unique values
}
for col in non_numeric_cols:
    
    missing_ratio = data[col].isna().mean() * 100

    num_values = data[col].nunique()

    cat_statistics["missing_ratio"].append(round(missing_ratio, 1))
    cat_statistics["num_values"].append(num_values)

    
cat_col_info_df = pd.DataFrame(cat_statistics, index=non_numeric_cols).T
print (cat_col_info_df)

               Poster_Link  Title  Overview  Certificate  Genre  Actors  \
missing_ratio          0.0    0.0       0.0         17.8    0.0     0.8   
num_values           500.0  499.0     500.0          6.0  254.0   494.0   

               Director  
missing_ratio       0.0  
num_values        359.0  


### => After confirming the percentage of missing values in the columns of the data, we will begin handling the missing data by using the TMDB API key to fill in the missing attributes
The missing values in the Certificate, rottenRating, and imdbRating columns can be explained as follows:
- The movie's certificate data might not be available on the TMDB website.
- The missing rottenRating and imdbRating scores are due to merging files based on movie titles. The original TMDB file contains 500 rows of data, while the other two files contain 300 and 250 rows, respectively. This discrepancy leads to missing scores.



## Filled missing data

In [26]:
# Fill missing values in numeric columns with 0 (imdbRating, rottenRating)
data[numeric_cols] = data[numeric_cols].fillna(0)
data.to_csv('merged_result.csv', index=False)

In [27]:
import requests
import csv
import time

API_KEY = '738f8682fc5143163b145d03a2016b0b'
BASE_URL = 'https://api.themoviedb.org/3'
SEARCH_URL = f'{BASE_URL}/search/movie'

def search_movie_by_title(title):
    params = {
        'api_key': API_KEY,
        'query': title,
        'language': 'en-US'
    }
    response = requests.get(SEARCH_URL, params=params)
    if response.status_code == 200:
        results = response.json().get('results', [])
        return results[0] if results else None
    return None

def get_movie_details(movie_id):
    url = f'{BASE_URL}/movie/{movie_id}?language=en-US&api_key={API_KEY}'
    return requests.get(url).json()

def get_movie_credits(movie_id):
    url = f'{BASE_URL}/movie/{movie_id}/credits?api_key={API_KEY}'
    return requests.get(url).json()

def get_movie_certification(movie_id):
    url = f'{BASE_URL}/movie/{movie_id}/release_dates?api_key={API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        results = response.json().get("results", [])
        for country in results:
            if country.get("iso_3166_1") == "US":
                for release in country.get("release_dates", []):
                    cert = release.get("certification")
                    if cert:
                        return cert
    return ''

def fill_missing_fields(row):
    title = row.get('Title', '')
    if not title:
        print("❌ Title is missing in the row")
        return row

    movie = search_movie_by_title(title)
    if not movie:
        print(f"❌ Movie not found: {title}")
        return row

    movie_id = movie['id']
    details = get_movie_details(movie_id)
    credits = get_movie_credits(movie_id)
    cert = get_movie_certification(movie_id)

    if not row.get('Poster_Link'):
        poster_path = details.get('poster_path', '')
        row['Poster_Link'] = f'https://image.tmdb.org/t/p/w500{poster_path}' if poster_path else ''

    if not row.get('Overview'):
        row['Overview'] = details.get('overview', '')

    if not row.get('Certificate'):
        row['Certificate'] = cert

    if not row.get('Runtime (min)'):
        row['Runtime (min)'] = details.get('runtime', 0)

    if not row.get('Genre'):
        genres = [genre['name'] for genre in details.get('genres', [])]
        row['Genre'] = ', '.join(genres)

    if not row.get('Actors'):
        actors = [cast['name'] for cast in credits.get('cast', [])[:5]]
        row['Actors'] = ', '.join(actors)

    if not row.get('Director'):
        directors = [crew['name'] for crew in credits.get('crew', []) if crew['job'] == 'Director']
        row['Director'] = ', '.join(directors)

    if not row.get('Year'):
        row['Year'] = details.get('release_date', '')[:4]

    if not row.get('tmdbRating'):
        row['tmdbRating'] = details.get('vote_average', 0)


    time.sleep(0.25)
    print(f"✅ Filled: {title}")
    return row

def main():
    input_file = 'merged_result.csv'
    data = []
    with open(input_file, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        data = list(reader)

    # Giữ nguyên phần xử lý tiếp theo
    filled_rows = []
    for row in data:
        needs_fill = any(not row.get(field) for field in [
            'Poster_Link', 'Overview', 'Certificate', 'Runtime (min)',
            'Genre', 'Actors', 'Director', 'Year', 'tmdbRating'
        ])
        if needs_fill:
            row = fill_missing_fields(row)
        filled_rows.append(row)

    fieldnames = [
        'Poster_Link', 'Title', 'Certificate', 'Overview',
        'Runtime (min)', 'Genre', 'Actors', 'Director',
        'Year', 'imdbRating', 'rottenRating', 'tmdbRating'
    ]

    output_file = 'movie.csv'
    with open(output_file, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for row in filled_rows:
            writer.writerow({field: row.get(field, '') for field in fieldnames})

    print("✅ Done! Output saved to:", output_file)


# Ví dụ sử dụng:
if __name__ == '__main__':
    main()
    

✅ Filled: A Dog's Will
✅ Filled: Impossible Things
✅ Filled: Gabriel's Inferno
✅ Filled: Hope
✅ Filled: Gabriel's Inferno: Part III
✅ Filled: Lucy Shimmers and the Prince of Peace
✅ Filled: The Quintessential Quintuplets Movie
✅ Filled: Josee, the Tiger and the Fish
✅ Filled: Dedicated to my ex
✅ Filled: We All Loved Each Other So Much
✅ Filled: Given
✅ Filled: Miracle in Cell No. 7
✅ Filled: I Want to Eat Your Pancreas
✅ Filled: Out of the Clear Blue Sky
✅ Filled: Il Sorpasso
✅ Filled: The Apartment
✅ Filled: Flow
✅ Filled: Michael Jackson's Thriller
✅ Filled: My Mom Is a Character 3
✅ Filled: Doctor Who: The Day of the Doctor
✅ Filled: 20th Century Girl
✅ Filled: Tokyo Story
✅ Filled: The Kid
✅ Filled: Persona
✅ Filled: Piper
✅ Filled: Capernaum
✅ Filled: Scenes from a Marriage
✅ Filled: Violet Evergarden: Eternity and the Auto Memory Doll
✅ Filled: Silenced
✅ Filled: Central Station
✅ Filled: Red Beard
✅ Filled: Wild Strawberries
✅ Filled: “The Shorts” by Aldo, Giovanni and Giacomo


### Check again after filled null values

In [28]:
data_new = pd.read_csv('movie.csv')

# Check if data have duplicate rows
num_duplicated_rows = data_new.duplicated().sum()
if num_duplicated_rows > 0:
    print(f"Found {num_duplicated_rows} duplicated rows. Removing duplicates...")
    data_new = data_new.drop_duplicates()
    print("✅ Duplicates removed.")
else:
    print("✅ Your data has no duplicated rows.")

def missing_ratio(column):
    return column.isna().mean() * 100

# Lấy tất cả các cột
all_cols = data_new.columns

# Function to calculate the percentage of missing values in each column
def calculate_missing_percentage(df):
    return df.isnull().mean() * 100

# Reset the statistics dictionary to avoid conflicts with previous data
statistics = {
    "missing_ratio": []
}

for col in all_cols:
    col_data = data_new[col]
    missing_ratio_val = missing_ratio(col_data)
    statistics["missing_ratio"].append(missing_ratio_val)

# Create a DataFrame from the statistics dictionary
col_info_df = pd.DataFrame(statistics, index=all_cols).T.round(1)
print(col_info_df)

✅ Your data has no duplicated rows.
               Poster_Link  Title  Certificate  Overview  Runtime (min)  \
missing_ratio          0.0    0.0         16.8       0.0            0.0   

               Genre  Actors  Director  Year  imdbRating  rottenRating  \
missing_ratio    0.0     0.6       0.0   0.0         0.0           0.0   

               tmdbRating  
missing_ratio         0.0  
